# LAB 4

Networks and Systems Security
Week 04
Malicious Software



## Aims of the Seminar

This workshop begins with a conceptual overview of the main categories of malware,
their propagation mechanisms, and payload behaviours. Then we move into a
series of hands-on Python exercises designed to illustrate how malware can be
detected, analysed, and contained using simple yet powerful tools.

Rather than handling real malicious code, the activities simulate key behaviours—file
integrity checking, signature scanning, and worm propagation—so participants can
safely explore how malware interacts with systems. 

By the end of the workshop, you
will have a clearer view of how malicious software functions, how it can be identified,
and how thoughtful countermeasures—both technical and human—can reduce its
impact.


Workshop Outline:
1. Explain the main categories of malware (virus, worm, trojan,
spyware, rootkit, etc.).
2. Understand how malware propagates and hides itself.
3. Demonstrate basic malware detection and analysis principles
using Python.
4. Apply file integrity checking and simple anomaly detection as
defensive countermeasures.

#### Resources
- hashlib Library
https://www.geeksforgeeks.org/python/hashlib-module-in-python/

## File Integrity Checker

- Understand how host-based antivirus systems detect unauthorized
file modifications by using file hashing as a baseline for system integrity

In [ ]:
# Script to loop through the files and generate a SHA256 hash for each. 
# The results should be saved in a csv file with the file name and the hash and time stamp. 

import sys
import hashlib
import os
import datetime
import csv

#function to hash files in a given directory
def hashfile(file, file_path):
    file_hashes = []
    BUF_SIZE = 65536
    sha256 = hashlib.sha256()

    for i in os.listdir(file_path):
        with open(os.path.join(file_path, i), 'rb') as f:
            while True:
                data = f.read(BUF_SIZE)

                if not data:
                    break

                sha256.update(data)
                file_hashes.append({"file": i, "hash": sha256.hexdigest(),"time": datetime.datetime.now()})

    return(file_hashes)

# file_path = input("Enter the path of the folder: ")
file_path = "C://Users//kierm//Desktop//Goldsmiths//Y3//Net and sys security//Workshop//4 Malicious Software//Data"

file_hashes = hashfile(sys.argv[1], file_path)
print(f"Hashes:{file_hashes}")

#witing the hashes to a csv file
with open("new-hashes.csv", "w", newline="") as csvfile:
    fieldnames = ["file", "hash", "time"]
    writer = csv.DictWriter(csvfile,
    fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(file_hashes)

Hashes:[{'file': '1 one.txt', 'hash': 'dd982210e11dcf0f614c64101ef75ac1eed4093f7d4d0b82bfed9170a21e9384', 'time': datetime.datetime(2025, 10, 22, 16, 59, 38, 313007)}, {'file': '2 two.txt', 'hash': 'a4db4924ec97969a3c1076a8464b30b4e6929545e7154b3f305b1d5228079870', 'time': datetime.datetime(2025, 10, 22, 16, 59, 38, 313007)}, {'file': '3 three.txt', 'hash': 'e462556f46d7b3dc4c1f21b289453ff760d7002fd03c9e672dbdd6f5b06828f2', 'time': datetime.datetime(2025, 10, 22, 16, 59, 38, 313007)}, {'file': '4 four.txt', 'hash': '3d7827fa75ec4a936681d2b70287bf849a92dd49850a2a1e2d7bfc455636813b', 'time': datetime.datetime(2025, 10, 22, 16, 59, 38, 318103)}, {'file': 'cheese.txt', 'hash': '4cd526b61f85b002234ed80e94808716bf2f75233c03f6db99e9f6f79281f047', 'time': datetime.datetime(2025, 10, 22, 16, 59, 38, 318318)}]


EXPLINATION OF SCRIPT

KEY FINDINGS

DESCUSSION POINTS

REAL LIFE SCENARIO

(Discussion Points
• Why do security systems rely on file hashes instead of timestamps
or file sizes?
• How might malware attempt to evade integrity checking?
• What happens if a legitimate system update changes many
hashes — how should you manage that?)

Write Python Code to Compare and Detect Changes


In [44]:
#function to detect changes between two csv files
def detect_changes(original_file, new_file):
    #load original hashes
    with open(original_file, "r") as csvfile:
        reader = csv.DictReader(csvfile)
        original_hashes = {row["file"]: row["hash"] for row in reader}

    #load new hashes
    with open(new_file, "r") as csvfile:
        reader = csv.DictReader(csvfile)
        new_hashes = {row["file"]: row["hash"] for row in reader}

    #detect changes
    changes = []
    for file, new_hash in new_hashes.items():
        original_hash = original_hashes.get(file)
        if original_hash is None:
            changes.append((file, "New file added"))
        elif original_hash != new_hash:
            changes.append((file, "File modified"))

    for file in original_hashes.keys():
        if file not in new_hashes:
            changes.append((file, "File deleted"))

    return changes


changes = detect_changes("hashes.csv", "new-hashes.csv")
for change in changes:
    print(f"{change[0]}: {change[1]}")

Write Python Code to Scan for Suspicious Patterns such as 
```bash
SIGNATURES = [r"eval\(", r"base64\.b64decode", r"socket\.connect", r"exec\(", 
r"import os"]
```


In [45]:
def scan_for_suspicious_patterns(file_path):
    SIGNATURES = [r"eval\(", r"base64\.b64decode", r"socket\.connect", r"exec\(", r"import os"]
    for i in os.listdir(file_path):
        with open(os.path.join(file_path, i), "r") as fp:
            lines = fp.readlines()
            for row in lines:
                for j in range(len(SIGNATURES)):
                    if row.find(SIGNATURES[j]) != -1:
                        print('String exists in file '+i)
                        print('Line Number:', lines.index(row))
                        print('Line Content:', row)

scan_for_suspicious_patterns(file_path)

import os
import re

def scan_for_suspicious_patterns(directory):
    pattern = re.compile(r"(eval\s*\(|base64\s*\.\s*b64decode|socket\s*\.\s*connect|exec\s*\(|import\s+os)", re.IGNORECASE)

    for root, _, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)

            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    for line_num, line in enumerate(f, 1):
                        if pattern.search(line):
                            print(f"[!] Suspicious pattern in {file_path}")
                            print(f"    Line {line_num}: {line.strip()}")
            except Exception as e:
                print(f"[Error] Could not read file: {file_path} ({e})")

# Example usage
scan_for_suspicious_patterns(file_path)


String exists in file 4 four.txt
Line Number: 3
Line Content: import os

[!] Suspicious pattern in C://Users//kierm//Desktop//Goldsmiths//Y3//Net and sys security//Workshop//4 Malicious Software//Data\4 four.txt
    Line 4: import os


Write Python Code to Simulate Network Propagation

In [ ]:
#network propagation simulation
def network_propagation_simulation():
    import socket
    import threading

    def handle_client(client_socket):
        request = client_socket.recv(1024)
        print(f"Received: {request.decode()}")
        client_socket.send(b"ACK")
        client_socket.close()

    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("WINDOWS_HOST", 9999))
    server.listen(5)
    print("Listening on WINDOWS_HOST:9999")
    while True:
        client, addr = server.accept()
        print(f"Accepted connection from {addr}")
        client_handler = threading.Thread(target=handle_client, args=(client,))
        client_handler.start()
# Uncomment the line below to run the network propagation simulation
# network_propagation_simulation()
